In [1]:
import pandas as pd
import numpy as np

In [3]:
# Read in the data
INPUT_DIR = './m5-forecasting-accuracy'
df_sale = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
df_calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
df_price = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
df_sample = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

In [4]:
# Forecast with MA
df_last_sale = df_sale.iloc[:, np.r_[0,-28:0]].melt('id', var_name='d', value_name='sale')
df_last_sale = df_last_sale.merge(df_calendar.loc[:,['d','date','wday']])
last_date = int(df_last_sale.d.max()[2:])
df_last_sale = df_last_sale.groupby(['id','wday'])['sale'].mean()


df_ma_forecast = df_sample.copy()
df_ma_forecast.columns = ['id'] + ['d_' + str(last_date + x) for x in range(1, 29)]
df_ma_forecast = df_ma_forecast.loc[df_ma_forecast.id.str.contains('validation')]

df_ma_forecast = df_ma_forecast.melt('id', var_name='d', value_name='sale')
df_ma_forecast = df_ma_forecast.drop('sale',axis = 1)
df_ma_forecast = df_ma_forecast.merge(df_calendar.loc[:,['d','date','wday']])
df_ma_forecast = df_ma_forecast.join(df_last_sale, on=['id', 'wday'])

df_ma_forecast = df_ma_forecast.pivot(index='id', columns='d', values='sale')
df_ma_forecast = df_ma_forecast.reset_index()

In [12]:
df_ma_forecast.to_csv("./submission_ma.csv")

In [11]:
df_ma_forecast.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,3.00,0.75,1.50,0.25,1.50,0.75,0.50,3.00,0.75,...,1.50,0.75,0.50,3.00,0.75,1.50,0.25,1.50,0.75,0.50
1,FOODS_1_001_CA_2_validation,0.25,0.50,1.25,1.00,0.25,0.00,4.50,0.25,0.50,...,0.25,0.00,4.50,0.25,0.50,1.25,1.00,0.25,0.00,4.50
2,FOODS_1_001_CA_3_validation,0.25,0.00,0.50,0.00,0.00,1.50,3.75,0.25,0.00,...,0.00,1.50,3.75,0.25,0.00,0.50,0.00,0.00,1.50,3.75
3,FOODS_1_001_CA_4_validation,1.25,0.00,0.25,0.00,0.25,0.25,0.25,1.25,0.00,...,0.25,0.25,0.25,1.25,0.00,0.25,0.00,0.25,0.25,0.25
4,FOODS_1_001_TX_1_validation,0.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.25,...,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00


In [10]:
df_ma_forecast.columns = df_sample.columns

In [ ]:
d